In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

iris = load_iris()
X = iris.data
y = iris.target

X_seq = X.reshape(X.shape[0], 4, 1)

encoder = OneHotEncoder(sparse_output=False)
y_onehot = encoder.fit_transform(y.reshape(-1, 1))

X_train, X_test, y_train, y_test = train_test_split(X_seq, y_onehot, test_size=0.2, random_state=42)

In [ ]:
class SimpleRNN:
    def __init__(self, input_size, hidden_size, output_size):
        self.hidden_size = hidden_size
        self.Wx = np.random.randn(hidden_size, input_size) * 0.01
        self.Wh = np.random.randn(hidden_size, hidden_size) * 0.01
        self.bh = np.zeros((hidden_size, 1))
        self.Wy = np.random.randn(output_size, hidden_size) * 0.01
        self.by = np.zeros((output_size, 1))

    def softmax(self, x):
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum(axis=0)

    def forward(self, inputs):
        h = np.zeros((self.hidden_size, 1))
        for t in range(len(inputs)):
            x_t = inputs[t].reshape(-1, 1)
            h = np.tanh(np.dot(self.Wx, x_t) + np.dot(self.Wh, h) + self.bh)
        y = np.dot(self.Wy, h) + self.by
        return self.softmax(y), h

    def predict(self, inputs):
        y, _ = self.forward(inputs)
        return np.argmax(y)

    def train(self, X, Y, epochs=100, lr=0.01):
        for epoch in range(epochs):
            loss = 0
            for i in range(len(X)):
                inputs = X[i]
                target = Y[i].reshape(-1, 1)

                h = np.zeros((self.hidden_size, 1))
                hs = [h]
                xs = []
                for t in range(len(inputs)):
                    x_t = inputs[t].reshape(-1, 1)
                    xs.append(x_t)
                    h = np.tanh(np.dot(self.Wx, x_t) + np.dot(self.Wh, h) + self.bh)
                    hs.append(h)

                y = np.dot(self.Wy, h) + self.by
                y_pred = self.softmax(y)

                loss += -np.sum(target * np.log(y_pred + 1e-9))

                dWy = np.zeros_like(self.Wy)
                dby = np.zeros_like(self.by)
                dWx = np.zeros_like(self.Wx)
                dWh = np.zeros_like(self.Wh)
                dbh = np.zeros_like(self.bh)

                dy = y_pred - target
                dWy += np.dot(dy, hs[-1].T)
                dby += dy

                dh = np.dot(self.Wy.T, dy)

                for t in reversed(range(len(inputs))):
                    h_t = hs[t+1]
                    h_prev = hs[t]
                    dtanh = (1 - h_t ** 2) * dh
                    dbh += dtanh
                    dWx += np.dot(dtanh, xs[t].T)
                    dWh += np.dot(dtanh, h_prev.T)
                    dh = np.dot(self.Wh.T, dtanh)

                self.Wx -= lr * dWx
                self.Wh -= lr * dWh
                self.bh -= lr * dbh
                self.Wy -= lr * dWy
                self.by -= lr * dby

            if (epoch + 1) % 10 == 0:
                print(f'Epoch {epoch+1}/{epochs}, Loss: {loss/len(X):.4f}')

In [ ]:
rnn = SimpleRNN(input_size=1, hidden_size=10, output_size=3)
rnn.train(X_train, y_train, epochs=100, lr=0.01)

Epoch 10/100, Loss: 0.8421
Epoch 20/100, Loss: 0.3629
Epoch 30/100, Loss: 0.1999
Epoch 40/100, Loss: 0.1356
Epoch 50/100, Loss: 0.1095
Epoch 60/100, Loss: 0.0966
Epoch 70/100, Loss: 0.0904
Epoch 80/100, Loss: 0.0881
Epoch 90/100, Loss: 0.0875
Epoch 100/100, Loss: 0.0856


In [4]:
def predict_iris(input_features):
    input_array = np.array(input_features).reshape(4, 1)
    pred_class = rnn.predict(input_array)
    return iris.target_names[pred_class]

sepal_length = 5.1
sepal_width = 3.5
petal_length = 1.4
petal_width = 0.2

example_input = [sepal_length, sepal_width, petal_length, petal_width]
predicted_class = predict_iris(example_input)
print(f"Predicted IRIS class: {predicted_class}")

Predicted IRIS class: setosa
